In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from scipy import stats
from keras import backend as K
from keras.callbacks import TensorBoard, EarlyStopping
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers import MaxPooling1D, UpSampling1D
from keras.layers import MaxPooling2D
from keras.layers import Flatten, Reshape
from keras.layers import Input, Dense, Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers import RepeatVector
from keras.layers.pooling import GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.models import Sequential
import numpy as np
import os
import matplotlib.pyplot as plt
import random as rand
import pandas as pd
import tensorflow
import keras
import statistics as st
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import p_f1_recall as metric

In [ ]:
results = []

In [ ]:
###3 HYPERPARAMETERS ####
batch_size, epochs, verbose = 200, 1, 1

group_size = 48
num_connected_nodes = 10
kernal_size = 6
num_filters = 64

In [ ]:
##### First proces unlabled data

data_u = pd.read_csv('fullest_unlabeled_data.csv')

### Remove unncessary columns
data_u = data_u.iloc[:,[3,4,5,7,8,9,10]]

### Standardize the features
def standardize(data,col_nums_to_standardize):
    for col in col_nums_to_standardize:
        data.iloc[:,col] = (data.iloc[:,col] - np.mean(data.iloc[:,col]))/np.std(data.iloc[:,col])
    return data

data_u = standardize(data_u,col_nums_to_standardize = range(6))


#### Now process the labeled data
data_train = pd.read_csv('final_training.csv')
data_test = pd.read_csv('final_testing.csv')

### Remove unncessary columns
data_train = data_train.iloc[:,[3,4,5,7,8,9,10,12]]  ## Add 13 if need to keep IDs ## Drop heading because of low mutual information
data_test = data_test.iloc[:,[3,4,5,7,8,9,10,12]]

### Standardize the features
data_train = standardize(data_train,col_nums_to_standardize = range(6))
data_test = standardize(data_test,col_nums_to_standardize = range(6))

In [ ]:
#Break features into groups of size 

def break_into_samples(data,group_size):
    l = []
    sep_ts = set(data.loc[:,'ID'])
    for ts in sep_ts:
        sub_data  = np.array(data.loc[data.loc[:,'ID']== ts,:])
        a = int(sub_data.shape[0]/group_size)
        if a != 0:
            sub_data = sub_data[:(a*group_size),:]
            sub_data = [np.split(sub_data,a)]
            l = l + sub_data
    data = np.vstack(l)
    data = data[:,:,:-1]
    return data

chunked_data_train = break_into_samples(data_train,group_size)
chunked_data_test = break_into_samples(data_test,group_size)
chunked_u_data = break_into_samples(data_u,group_size)

In [ ]:
def split_train_val(data,labeled = True,thresh = 0.8):
    np.random.shuffle(data)
    thresh1 = int(data.shape[0]*thresh)
    training = data[:thresh1,:,:]
    val = data[thresh1:,:,:]
    return training, val


global_num_classes = -1

def split_feat_lab(data,many_to_one = True, prop = 0.65):
    features = data[:,:,:data.shape[2]-1]
    labels = data[:,:,data.shape[2]-1]
    if many_to_one == True:
        y_temp = np.zeros(labels.shape[0])
        y_temp.fill(np.nan)
        for i in range(labels.shape[0]):
            row = labels[i,:]
            trans_class = (Counter(row)).most_common(1)[0]
            if trans_class[1] < group_size*prop : 
                 y_temp[i] = 4
            else:
                 y_temp[i] = st.mode(row)
        labels = y_temp
        global global_num_classes
    if global_num_classes < 0:
        global_num_classes = len(set(labels.flatten('C')))
    num_classes = global_num_classes
    labels = to_categorical(labels,num_classes)
    return features,labels

In [ ]:
chunked_data_train1, chunked_data_val = split_train_val(chunked_data_train)
training, val = split_train_val(chunked_u_data)

train_x,train_y = split_feat_lab(chunked_data_train1)
val_x,val_y = split_feat_lab(chunked_data_val)
test_x,test_y = split_feat_lab(chunked_data_test)

In [ ]:
### CNN auto-encoder ###

n_timesteps = training.shape[1]
n_features = training.shape[2]
    
def build_autoencoder():
    raw_input = Input(shape=(n_timesteps,n_features))  
    encoding_1 = Conv1D(filters = num_filters, kernel_size = kernal_size, activation='relu', padding='same')(raw_input)
    encoding_2 = Conv1D(filters = num_filters, kernel_size = kernal_size, activation='relu', padding='same')(encoding_1)
    encoding_3 = MaxPooling1D(3, padding='same')(encoding_2)
    encoding_4 = Conv1D(filters = num_filters, kernel_size = kernal_size, activation='relu', padding='same')(encoding_3)
    encoding_5 = Conv1D(filters = num_filters, kernel_size = kernal_size, activation='relu', padding='same')(encoding_4)
    encoding_6 = MaxPooling1D(2)(encoding_5)
    encoded = Dropout(0.5)(encoding_6)
    decoding_1 = Conv1D(filters = num_filters, kernel_size = kernal_size, activation='relu', padding='same')(encoded)
    decoding_2 = Conv1D(filters = num_filters, kernel_size = kernal_size, activation='relu', padding='same')(decoding_1)
    decoding_3 = UpSampling1D(size = 2)(decoding_2)
    decoding_4 = Conv1D(filters = num_filters, kernel_size = kernal_size, activation='relu', padding='same')(decoding_3)
    decoding_5 = Conv1D(filters = n_features, kernel_size = kernal_size, activation='relu', padding='same')(decoding_4)
    decoded = UpSampling1D(size = 3)(decoding_5)
    encoder = Model(raw_input, encoded)
    autoencoder = Model(raw_input, decoded)
    return autoencoder, encoder
    
autoencoder, encoder = build_autoencoder()
init_weights = encoder.get_weights()

def autoencoder_train(autoencoder):
    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.fit(training, training,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(val, val),
                callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)])
    loss = autoencoder.evaluate(val, val, batch_size=batch_size, verbose=verbose)
    return loss

autoencoder_loss = autoencoder_train(autoencoder)

trained_weights = encoder.get_weights()

Train on 116447 samples, validate on 29112 samples
Epoch 1/1
29112/29112 [==============================] - 4s 145us/step


In [ ]:
encoder.set_weights(trained_weights)
new_layer1 = Flatten()(encoder.output) ## Could try GlobalAveragePool here instead
new_layer2 = Dense(num_connected_nodes, activation = 'relu')(new_layer1)
new_layer3 = Dense(global_num_classes, activation='softmax')(new_layer2)
pre_trained = Model(encoder.input,new_layer3)
pre_trained.summary()

def train_model(model):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',metric.recall,metric.precision,metric.f1])
    model.fit(train_x,train_y,epochs=epochs,batch_size=batch_size,verbose=verbose,validation_data=(val_x,val_y),
             callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)])
    loss,accuracy,recall_score,precision_score,f_score = model.evaluate(val_x, val_y, batch_size=batch_size, verbose=verbose)
    #final_loss,final_acc = model.evaluate(test_features_x, test_features_y, batch_size = batch_size, verbose=verbose)
    return loss,accuracy,recall_score,precision_score,f_score # . ,final_loss,final_acc

m1_loss,m1_accuracy,m1_recall_score,m1_precision_score,m1_f_score = train_model(pre_trained)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 48, 6)             0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 48, 64)            2368      
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 48, 64)            24640     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 16, 64)            0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 16, 64)            24640     
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 16, 64)            24640     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 8, 64)             0         
__________

(1.2075842991777899,
 0.6049382843598416,
 0.2757201660807731,
 0.779182188304854,
 0.40719928270504796)

In [ ]:
encoder.set_weights(init_weights)
new_layer1 = Flatten()(encoder.output) ## Could try GlobalAveragePool here instead
new_layer2 = Dense(num_connected_nodes, activation = 'relu')(new_layer1)
new_layer3 = Dense(global_num_classes, activation='softmax')(new_layer2)
no_pre_training = Model(encoder.input,new_layer3)  ## This just makes a model without the encoding

m2_loss,m2_accuracy,m2_recall_score,m2_precision_score,m2_f_score = train_model(no_pre_training)

Train on 969 samples, validate on 243 samples
Epoch 1/1
243/243 [==============================] - 0s 100us/step


(1.3068794792081102,
 0.4855967179983241,
 0.2716049436679102,
 0.6665426953829856,
 0.38592939428341244)

In [ ]:
results.append([autoencoder_loss,m1_f_score,m2_f_score])
print(np.mean(results,axis=0))
print(np.sd(results,axis=0))